https://mlang.frama.io/cours-marc-lang/stable/sigmaM2_telea/sigmaM2_projet.html#construire-un-masque-%C3%A0-partir-de-la-bd-for%C3%AAt

# **Construire un masque à partir de la BDFORET**


### Objectif
Le but est de produire un masque raster qui indique les zones à classer en forêt et les zones à ne pas classer, à partir des polygones de la **BD forêt**.

### Critères de classification
- **Zones à classer** : Ce sont les polygones de la BD forêt, à l'exception de :
  - **Lande** et **Formation Herbacée** (qui ne sont pas de la forêt),
  - **Forêt ouverte**,
  - **Forêt fermée sans couvert arboré**.
  
### Paramètres du masque
Le masque doit être conforme aux spécifications suivantes :

- **Format** : GeoTIFF
- **Encodage** : 8 bits
- **Emprise et résolution spatiale** : Doit correspondre à celles des images **S2** utilisées (après découpe, cf. section Pré-traitement des images)
- **Nom du fichier** : `masque_foret.tif`
- **Chemin de sortie** : `results/data/img_pretraitees`
- **Valeurs du masque** :
  - **Zone de forêt** : 1
  - **Zone hors forêt** : 0

### Exemple de valeurs dans le fichier raster
| Classe            | Valeur du pixel |
|-------------------|-----------------|
| Zone de forêt     | 1               |
| Zone hors forêt   | 0               |

In [25]:
# Chemins d'accès

shapefile_path = '../../data/project/FORMATION_VEGETALE.shp'
s2_image_path = '../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B11.tif'
output_path = '../results/data/img_pretraitees/masque_foret.tif'

In [26]:
# Import bibliothèques

import geopandas as gpd
from osgeo import gdal, ogr
import numpy as np

In [27]:
# Charger le shapefile
gdf = gpd.read_file(shapefile_path)

# Définir les valeurs à exclure
excluded_values = ['Formation herbacée', 'Forêt ouverte de conifères purs', 
                   'Forêt ouverte de feuillus purs', 'Forêt ouverte sans couvert arboré', 
                   'Forêt ouverte à mélange de feuillus et conifères', 'Lande']

# Ajouter la colonne "mask" qui prend valeur 1 ou 0 selon si forêt ou hors forêt
gdf['mask'] = gdf['TFV'].apply(lambda x: 0 if x in excluded_values else 1)

print(gdf.head())  # Vérifier si le champ 'mask' est correctement ajouté

print(gdf['mask'].value_counts())
print(gdf.crs)

                         ID CODE_TFV                             TFV  \
0  FORESTIE0000000003100001      FO1  Forêt ouverte de feuillus purs   
1  FORESTIE0000000003100002      FO1  Forêt ouverte de feuillus purs   
2  FORESTIE0000000003100003      FO1  Forêt ouverte de feuillus purs   
3  FORESTIE0000000003100004      FO1  Forêt ouverte de feuillus purs   
4  FORESTIE0000000003100005      FO1  Forêt ouverte de feuillus purs   

                  TFV_G11   ESSENCE  \
0  Forêt ouverte feuillus  Feuillus   
1  Forêt ouverte feuillus  Feuillus   
2  Forêt ouverte feuillus  Feuillus   
3  Forêt ouverte feuillus  Feuillus   
4  Forêt ouverte feuillus  Feuillus   

                                            geometry  mask  
0  POLYGON ((508756.27 6181000.546, 508761.718 61...     0  
1  POLYGON ((504437.854 6181272.919, 504437.135 6...     0  
2  POLYGON ((508679 6181464, 508680.8 6181463, 50...     0  
3  POLYGON ((509217.516 6181694.51, 509213.5 6181...     0  
4  POLYGON ((508056 6181920

-------------------

https://mlang.frama.io/cours-marc-lang/stable/sigmaM2_telea/sigmaM2_labworks_write_and_load.html

In [28]:
# Ouvrir l'image
image_s2 = gdal.Open(s2_image_path)

# Obtenir la géotransformation
geotransform = image_s2.GetGeoTransform()
print("Géotransformation:", geotransform)

projection = image_s2.GetProjection()

print(projection)

# Extraire les valeurs de la géotransformation
raster_minx = geotransform[0]  # Coordonnée X du coin supérieur gauche
raster_maxy = geotransform[3]  # Coordonnée Y du coin supérieur gauche
raster_resolution_x = geotransform[1]  # Résolution en X
raster_resolution_y = -geotransform[5]  # Résolution en Y (valeur négative pour l'axe Y descendant)

# Obtenir la taille de l'image (en pixels)
raster_x_size = image_s2.RasterXSize  # Nombre de pixels en X
raster_y_size = image_s2.RasterYSize  # Nombre de pixels en Y

# Calculer l'emprise
raster_maxx = raster_minx + raster_resolution_x * raster_x_size
raster_miny = raster_maxy - (raster_resolution_y * raster_y_size)

Géotransformation: (300000.0, 20.0, 0.0, 4900020.0, 0.0, -20.0)
PROJCS["WGS 84 / UTM zone 31N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",3],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32631"]]


In [31]:
gdf = gdf.to_crs(projection)

print(gdf.head())

# Sauvegarder le GeoDataFrame modifié en tant que shapefile temporaire
temp_shapefile_path = '../results/data/img_pretraitees/formation_vegetale_mask.shp'
gdf.to_file(temp_shapefile_path)

                         ID CODE_TFV                             TFV  \
0  FORESTIE0000000003100001      FO1  Forêt ouverte de feuillus purs   
1  FORESTIE0000000003100002      FO1  Forêt ouverte de feuillus purs   
2  FORESTIE0000000003100003      FO1  Forêt ouverte de feuillus purs   
3  FORESTIE0000000003100004      FO1  Forêt ouverte de feuillus purs   
4  FORESTIE0000000003100005      FO1  Forêt ouverte de feuillus purs   

                  TFV_G11   ESSENCE  \
0  Forêt ouverte feuillus  Feuillus   
1  Forêt ouverte feuillus  Feuillus   
2  Forêt ouverte feuillus  Feuillus   
3  Forêt ouverte feuillus  Feuillus   
4  Forêt ouverte feuillus  Feuillus   

                                            geometry  mask  
0  POLYGON ((309032.524 4730491.765, 309037.965 4...     0  
1  POLYGON ((304718.567 4730755.446, 304717.85 47...     0  
2  POLYGON ((308954.455 4730954.532, 308956.254 4...     0  
3  POLYGON ((309491.904 4731185.805, 309487.913 4...     0  
4  POLYGON ((308331.3 47314

----------------

https://mlang.frama.io/cours-marc-lang/stable/sigmaM2_telea/sigmaM2_labworks_classifp1.html#ex-2

In [32]:
import os

# Définir les paramètres
field_name = 'mask'  # Utiliser le champ 'mask' comme champ pour la valeur des pixels

# Définir la résolution spatiale et l'emprise
spatial_resolution = raster_resolution_x


gdf_path = temp_shapefile_path 

# Définir le chemin du fichier de sortie
output_path = output_path  # Remplacez par un chemin valide

# Définir le modèle de commande (en ajoutant des guillemets autour des chemins)
cmd_pattern = (
    "gdal_rasterize -a {field_name} "
    "-tr {spatial_resolution} {spatial_resolution} "
    "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
    '"{in_vector}" "{out_image}"'
)

# Formater la commande avec les paramètres réels
cmd = cmd_pattern.format(
    in_vector=gdf_path, xmin=raster_minx, ymin=raster_miny, xmax=raster_maxx,
    ymax=raster_maxy, out_image=output_path, field_name=field_name,
    spatial_resolution=spatial_resolution
)

# Afficher la commande pour vérification
print("Exécution de la commande :", cmd)

# Exécuter la commande dans le terminal
os.system(cmd)


Exécution de la commande : gdal_rasterize -a mask -tr 20.0 20.0 -te 300000.0 4790220.0 409800.0 4900020.0 -ot Byte -of GTiff "../results/data/img_pretraitees/formation_vegetale_mask.shp" "../results/data/img_pretraitees/masque_foret.tif"
0

ERROR 1: PROJ: proj_identify: Open of /opt/conda/share/proj failed


...10...20...30...40...50...60...70...80...90...100 - done.


0